In [25]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

### Relevante Parameter:

array_phi = 1 / (1 + np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat)))

array_lambdaY = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))

    if r[0] > 0:
        for i in range(len(y_monat)):
            for j in range(y_monat[i] + 1):
                if j == 0:
                    pmf = phi[i] + (1 - phi[i]) * (1 + lambdaY[i] / r[0])**(-r[0])
                elif j > 0:
                    numerator = math.gamma(j + r[0])
                    denominator = math.factorial(j) * math.gamma(r[0])
                    gamma_term = numerator / denominator
                    pmf= (1 - phi[i]) * gamma_term * (1 + lambdaY[i] / r[0])**(-r[0]) * (1 + r[0] / lambdaY[i])**(-j)
                cdf[i] += pmf
    return cdf

Die Öffungszahlen sind mit einer ZINBD modelliert und die Werte folgender Tabellen haben Einfluss:

1. df_LY
2. LO_monat
3. df_trans_werte
4. df_HY

folgende Parameter haben einen Einfluss:
1. delta_0 (1 x anzahl_states array, also state abhängig)
2. delta_1 (1 x anzahl_states array, also state abhängig)

3. alpha_p (1 x anzahl_states array, also state abhängig)
4. beta_p_1 (1 x anzahl_states array, also state abhängig)
5. beta_p_1 (1 x anzahl_states array, also state abhängig)



(1 x 1 array, state unabhängig)

In [26]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY = pd.read_csv(os.path.join(path, filename))

In [27]:
# LO monat
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

In [28]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_trans_werte = pd.read_csv(os.path.join(path, filename))

In [29]:
# EM_monat aus der HY Zahlen der versendeten Mails ziehen
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

### Datenanpassung wie bei der Berechnung der optimalen Parameter

In [30]:
## Alle DataFrame Werte von Float zu int
df_LY = df_LY.astype(int)
df_LO = df_LO.astype(int)
df_trans_werte = df_HY.astype(int)
df_HY = df_LY.astype(int)

### Sicherstellen dass alle df dieselben Kunden verwenden

In [31]:
# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LY = df_LY[df_LY['iid'].isin(df_LO['iid'])]
df_LO = df_LO[df_LO['iid'].isin(df_trans_werte['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_HY['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LY['iid'])]

In [32]:
len(df_LY)

647

In [33]:
len(df_LO)

647

In [34]:
len(df_trans_werte)

697

In [35]:
len(df_HY)

647

In [36]:
# Siherstellen dass alle dataframes die Kunden in derselben Reihenfolge auflisten
df_LY = df_LY.sort_values('iid')
df_LO = df_LO.sort_values('iid')
df_trans_werte = df_trans_werte.sort_values('iid')
df_HY = df_HY.sort_values('iid')

#Index resetten
df_LY = df_LY.reset_index(drop = True)
df_LO = df_LO.reset_index(drop = True)
df_trans_werte = df_trans_werte.reset_index(drop = True)
df_HY = df_HY.reset_index(drop = True)

### Ausreißer bei den Kunden löschen (wie bei den Berechnungen)

In [37]:
kunden_pos = [443, 387, 408, 415, 451, 531, 364, 351, 348, 501, 408, 285, 422, 251, 220]

In [38]:
# Zeilen löschen
df_LY = df_LY.drop(kunden_pos).reset_index(drop = True)
df_LO = df_LO.drop(kunden_pos).reset_index(drop = True)
df_trans_werte = df_trans_werte.drop(kunden_pos).reset_index(drop = True)
df_HY = df_HY.drop(kunden_pos).reset_index(drop = True)

### Spezifikation des Modells

In [39]:
anzahl_kunden = len(df_LY["iid"])
anzahl_kunden

633

In [40]:
anzahl_monate = len(df_LY.columns[df_LY.columns != 'iid'])
anzahl_monate

15

In [41]:
anzahl_states = 3

### Nelder-Mead mit poisite Wahresceinlichkeiten nach state 1, 30k Berechungen

### 1. Herauslesen der Parameter aus dem array der gesamten geschätzten Parameter

In [42]:
estimated_params=[ 0.00000000e+00, -1.08065873e+01,  1.86319601e+01,  0.00000000e+00,
       -2.46579307e+01,  7.23046373e-01,  0.00000000e+00, -3.62911892e+00,
        1.70149354e+01,  0.00000000e+00,  1.22702980e+00, -2.31363378e+00,
        0.00000000e+00,  1.03831097e+00,  3.98501866e+00,  0.00000000e+00,
       -1.03933137e+00, -8.77566481e-01,  0.00000000e+00,  1.76273515e+00,
       -7.10962468e-01,  0.00000000e+00, -8.89512765e+00, -4.63643365e-01,
        0.00000000e+00,  4.95599353e+00, -9.10157573e-01,  0.00000000e+00,
        3.03642274e+00, -5.63131592e+00,  0.00000000e+00,  4.61318628e-01,
        1.15359884e+01,  0.00000000e+00, -1.57901445e-01,  2.92499573e+00,
        0.00000000e+00, -1.14498657e+00,  2.40136701e+00,  0.00000000e+00,
        9.19527616e-01,  2.09920057e+01,  0.00000000e+00, -9.65897727e-02,
       -7.85542900e-01,  1.09629299e-01,  3.29483116e+01,  1.57798185e-01,
       -6.12830973e-01,  2.22484973e+00, -1.07644136e+01,  6.86142125e-03,
        0.00000000e+00, -1.18898698e+00, -2.74512446e-01,  2.20371399e+00,
       -2.17264903e+01,  8.45242511e-01, -1.45726730e-01,  1.31132997e+01,
        2.68518247e-01,  8.92791031e-02, -2.06771370e+00, -1.52907385e-01,
        1.14177604e+00]

In [43]:
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4

alpha = estimated_params[:alpha_size]
gamma = estimated_params[alpha_size:alpha_size+gamma_size]
alpha_o = estimated_params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
beta_o = estimated_params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
delta_0 = estimated_params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
delta_1 = estimated_params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
alpha_p = estimated_params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
beta_p_1 = estimated_params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
beta_p_2 = estimated_params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
theta = estimated_params[alpha_size+gamma_size+anzahl_states*6+1:]

In [44]:
delta_0

[2.22484973, -10.7644136, 0.00686142125]

In [45]:
delta_1

[0.0, -1.18898698, -0.274512446]

In [46]:
alpha_p

[2.20371399, -21.7264903, 0.845242511]

In [47]:
beta_p_1

[-0.14572673, 13.1132997, 0.268518247]

In [48]:
beta_p_2

[0.0892791031, -2.0677137, -0.152907385]

### Erstellen der erwarteten Käufe in den jeweiligen Monaten

In [49]:
# Erwartungswert = [r * (1-p)] / p

#### r, die Anzahl der Erfolge bis zum Abbruch, wurde vorher geschätzt

In [60]:
#r = 0.779

#### Berechnun von p

In [59]:
def p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    LY_monat = df_LY[str_monat]
    
    array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
    array_p = array_exp / (1+ array_exp)
    
    return array_p

### Berechung von r'

In [61]:
def r_state_monat(delta_0, delta_1, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    LY_monat = df_LY[str_monat]
    array_r = np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat))
    
    return array_r

#### Berechnung der Erwartungswerte

In [62]:
# Speichern der Erwartungswerte in einem array
erwartung = np.zeros((anzahl_kunden, anzahl_states, anzahl_monate))

In [63]:
for state in range(1, anzahl_states + 1):
        for monat in range(1, anzahl_monate +1):
            p_array = p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
            array_r = r_state_monat(delta_0, delta_1, state, monat)
            erwartung[:, state-1, monat-1] = (array_r * (1-p_array) / p_array)

In [64]:
np.sum(erwartung[:,0,:])/erwartung[:,0,:].size

0.7211761595224049

In [65]:
np.sum(erwartung[:,1,:])/erwartung[:,1,:].size

3.487488654647268e+96

In [66]:
np.sum(erwartung[:,2,:])/erwartung[:,2,:].size

41837471.56124341

In [67]:
p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, 2, 1)

0      0.000023
1      0.000023
2      0.000023
3      0.000023
4      0.000023
         ...   
628    0.000023
629    0.000023
630    0.000023
631    0.000023
632    0.000023
Name: 1, Length: 633, dtype: float64

In [68]:
df_HY["1"]

0      1
1      1
2      1
3      1
4      1
      ..
628    1
629    1
630    1
631    1
632    1
Name: 1, Length: 633, dtype: int32